# Лабораторная работа: Деревья и двоичная куча

В этой расширенной лабораторной работе вы изучите различные виды деревьев, реализуете двоичную **max-кучу** и решите практические задачи с её использованием.

## Цели
- Изучить различные виды древовидных структур данных
- Реализовать N-арные деревья, BST и Trie
- Освоить двоичную кучу и её основные операции
- Применить кучи для решения практических алгоритмических задач
- Сравнить производительность различных структур данных

In [ ]:
import random
import time
import heapq
from typing import List, Optional

# Для визуализации
try:
    from graphviz import Digraph
except ImportError:
    %pip install graphviz --quiet
    from graphviz import Digraph

## Часть 1: Виды деревьев

Изучим различные виды древовидных структур данных и их применения.

### Задание 1.1: N-арное дерево
Реализуйте N-арное дерево, где каждый узел может иметь произвольное количество потомков.

In [3]:
class NaryNode:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        """Добавить потомка"""
        # TODO: Добавьте ребенка в список children
        self.children.append(child)

    def find_child(self, value):
        """Найти потомка по значению"""
        # TODO: Найти и вернуть потомка с заданным значением
        for child in self.children:
            if child.value == value:
                return child
            found = child.find_child(value)
            if found:
                return found
            return None
            

    def print_tree(self, level=0):
        """Печать дерева с отступами"""
        # TODO: Рекурсивная печать дерева с отступами
        print("  " * level + "- " + str(self.value))
        for child in self.children:
            child.print_tree(level + 1)

# Тест N-арного дерева
root = NaryNode('A')
root.add_child(NaryNode('B'))
root.add_child(NaryNode('C'))
root.add_child(NaryNode('D'))
root.children[0].add_child(NaryNode('E'))
root.children[0].add_child(NaryNode('F'))

print("N-арное дерево:")
root.print_tree()
print("Поиск 'E':", root.find_child('E').value if root.find_child('E') else "Не найдено")

N-арное дерево:
- A
  - B
    - E
    - F
  - C
  - D
Поиск 'E': E


### Задание 1.2: Префиксное дерево (Trie)
Реализуйте Trie для эффективного поиска строк.

In [2]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        """Вставить слово в Trie"""
        # TODO: Добавьте слово в дерево
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, word):
        """Найти слово в Trie"""
        # TODO: Проверьте наличие полного слова
        node = self._traverse(word)
        return node is not None and node.is_end_of_word

    def starts_with(self, prefix):
        """Проверить, есть ли слова с данным префиксом"""
        # TODO: Проверьте наличие префикса
        return self._traverse(prefix) is not None
    
    def _traverse(self, text):
        """Вспомогательная функция для прохода по Trie"""
        node = self.root
        for char in text:
            if char not in node.children: 
                return None
            node = node.children[char]
        return node

# Тест Trie
trie = Trie()
words = ["hello", "world", "word", "hi", "hey"]
for word in words:
    trie.insert(word)

print("Поиск 'hello':", trie.search("hello"))
print("Поиск 'hel':", trie.search("hel"))
print("Префикс 'he':", trie.starts_with("he"))
print("Префикс 'wor':", trie.starts_with("wor"))

Поиск 'hello': True
Поиск 'hel': False
Префикс 'he': True
Префикс 'wor': True


## Часть 2: Двоичная куча

### Теория
Двоичная куча — это почти полное бинарное дерево, удовлетворяющее свойству кучи: ключ каждого узла \(\ge\) ключей потомков (для max-кучи). Основные операции выполняются за \(O(\log n)\) благодаря ограниченной высоте дерева.

In [4]:
import random
from typing import List

In [6]:
class MaxHeap:
    def __init__(self):
        self.heap: List[int] = []
  
    def _get_parent_idx(self, idx: int) -> int:
        return (idx - 1) // 2
    
    def _get_left_child_idx(self, idx: int) -> int:
        return 2 * idx + 1
    
    def _get_right_child_idx(self, idx: int) -> int:
        return 2 * idx + 2

    def _sift_up(self, idx: int) -> None:
        # TODO: Реализуйте просеивание вверх
        current_idx = idx
        parent_idx = self._get_parent_idx(current_idx)

        while current_idx > 0 and self.heap[current_idx] > self.heap[parent_idx]:
            self.heap[current_idx], self.heap[parent_idx] = self.heap[parent_idx], self.heap[current_idx]
            current_idx = parent_idx
            parent_idx = self._get_parent_idx(current_idx)


    def _sift_down(self, idx: int) -> None:
        # TODO: Реализуйте просеивание вниз
        n = len(self.heap)
        current_idx = idx
        
        while True:
            left_idx = self._get_left_child_idx(current_idx)
            right_idx = self._get_right_child_idx(current_idx)
            largest = current_idx

            if left_idx < n and self.heap[left_idx] > self.heap[largest]:
                largest = left_idx
            
            if right_idx < n and self.heap[right_idx] > self.heap[largest]:
                largest = right_idx

            if largest != current_idx:
                self.heap[current_idx], self.heap[largest] = self.heap[largest], self.heap[current_idx]
                current_idx = largest
            else:
                break

    def insert(self, value: int) -> None:
        """Добавить элемент в кучу"""
        # TODO: Добавить значение и вызвать _sift_up
        self.heap.append(value)
        self._sift_up(len(self.heap) - 1)

    def extract_max(self):
        """Извлечь максимальный элемент из кучи"""
        # TODO: Реализовать удаление корня и восстановление кучи
        if not self.heap:
            return None
        if len(self.heap) == 1:
            return self.heap.pop()
        
        max_val = self.heap[0]
        self.heap[0] = self.heap.pop()
        self._sift_down(0)
        return max_val

    def heapify(self, arr: List[int]):
        """Построить кучу из массива за O(n)"""
        self.heap = arr[:]
        # TODO: Реализуйте построчный вызов _sift_down начиная с последнего внутреннего узла
        n = len(self.heap)
        start_idx = self._get_parent_idx(n - 1)
        
        for i in range(start_idx, -1, -1):
            self._sift_down(i)
    
    def peek(self):
        """Вернуть максимальный элемент без удаления"""
        return self.heap[0] if self.heap else None


    def __len__(self):
        return len(self.heap)

    def __repr__(self):
        return str(self.heap)

In [7]:
# Тесты базовой кучи
heap = MaxHeap()
for x in [50, 20, 70, 80, 60, 25, 90]:
    heap.insert(x)
print('Куча:', heap)
print('Максимум:', heap.extract_max())
print('После извлечения:', heap)

Куча: [90, 70, 80, 20, 60, 25, 50]
Максимум: 90
После извлечения: [80, 70, 50, 20, 60, 25]


In [13]:
def heapsort(arr: List[int]) -> List[int]:
    """Сортировка кучей (max-heap). Верните новый отсортированный список."""
    # TODO: Реализуйте heapsort, используя методы вашего класса
    heap_obj = MaxHeap()
    heap_obj.heapify(arr)
    n = len(arr)

    sorted_arr = []
    while len(heap_obj) > 0:
        sorted_arr.append(heap_obj.extract_max())
    return sorted_arr

# Проверьте на случайном массиве
arr = random.sample(range(1000), 20)
print('Исходный:', arr)
print('Отсортированный:', heapsort(arr))

Исходный: [387, 212, 812, 32, 680, 444, 256, 701, 467, 655, 517, 842, 315, 872, 817, 850, 91, 919, 805, 399]
Отсортированный: [919, 872, 850, 842, 817, 812, 805, 701, 680, 655, 517, 467, 444, 399, 387, 315, 256, 212, 91, 32]


## Часть 3: Практические задачи с использованием кучи

Теперь применим кучу для решения реальных алгоритмических задач.

### Задание 3.1: K наибольших элементов
Найдите K наибольших элементов в массиве за O(n log k).

In [15]:
import heapq
from typing import List

def find_k_largest_heap(arr, k):
    """Найти k наибольших элементов используя min-heap размера k"""
    # TODO: Используйте min-heap для поддержания k наибольших элементов
    # Подсказка: используйте heapq (min-heap) и поддерживайте размер k
    if k == 0:
        return []
        

    min_heap = []
    
    for num in arr:
        if len(min_heap) < k:
            heapq.heappush(min_heap, num)
        elif num > min_heap[0]:
            heapq.heappop(min_heap)
            heapq.heappush(min_heap, num)
            
    return min_heap


def find_k_largest_manual(arr, k):
    """Найти k наибольших элементов используя вашу реализацию MaxHeap"""
    # TODO: Используйте вашу MaxHeap для решения задачи
    if k == 0:
        return []
        
    #MaxHeap
    heap_obj = MaxHeap()
    heap_obj.heapify(arr)

    result = []
    for _ in range(k):
        max_val = heap_obj.extract_max()
        if max_val is not None:
            result.append(max_val)
        else:
            break
            
    return result

# Тест
test_array = [3, 7, 1, 9, 4, 6, 8, 2, 5]
k = 3

print("Исходный массив:", test_array)
print("3 наибольших (heapq):", find_k_largest_heap(test_array, k))
print("3 наибольших (MaxHeap):", find_k_largest_manual(test_array, k))

Исходный массив: [3, 7, 1, 9, 4, 6, 8, 2, 5]
3 наибольших (heapq): [7, 9, 8]
3 наибольших (MaxHeap): [9, 8, 7]


### Задание 3.2: Медиана в потоке данных
Реализуйте структуру данных для нахождения медианы в потоке чисел.

In [16]:
import heapq
from typing import List

class MedianFinder:
    """Найти медиану в потоке данных используя две кучи"""

    def __init__(self):
        # TODO: Инициализируйте две кучи:
        # max_heap - для меньшей половины (используйте отрицательные числа)
        self.max_heap: List[int] = [] 
        # min_heap - для большей половины
        self.min_heap: List[int] = []

    def add_number(self, num):
        """Добавить число в поток"""
        # TODO: Добавьте число в соответствующую кучу
        # и балансируйте размеры куч
        if not self.max_heap or num <= -self.max_heap[0]:
            heapq.heappush(self.max_heap, -num) # отриц
        else:
            heapq.heappush(self.min_heap, num)
            
        # баланс
        
        if len(self.max_heap) > len(self.min_heap) + 1:
            max_val = heapq.heappop(self.max_heap)
            heapq.heappush(self.min_heap, -max_val)
            
        elif len(self.min_heap) > len(self.max_heap):
            min_val = heapq.heappop(self.min_heap)
            heapq.heappush(self.max_heap, -min_val)

    def find_median(self):
        """Найти медиану текущего потока"""
        # TODO: Вычислите медиану на основе размеров и вершин куч
        total_size = len(self.max_heap) + len(self.min_heap)
        if total_size == 0:
            return 0.0 
        
        if total_size % 2 == 1:
            if len(self.max_heap) > len(self.min_heap):
                # max heap больше
                return float(-self.max_heap[0])
            else:
                # max hwap меньше
                return float(self.min_heap[0]) 
        else:
            median = (-self.max_heap[0] + self.min_heap[0]) / 2.0
            return median

# Тест MedianFinder
mf = MedianFinder()
numbers = [1, 5, 2, 6, 3, 4]
for num in numbers:
    mf.add_number(num)
    print(f"Добавлено {num}, max_heap: {[-x for x in mf.max_heap]}, min_heap: {mf.min_heap}, медиана: {mf.find_median()}")

Добавлено 1, max_heap: [1], min_heap: [], медиана: 1.0
Добавлено 5, max_heap: [1], min_heap: [5], медиана: 3.0
Добавлено 2, max_heap: [2, 1], min_heap: [5], медиана: 2.0
Добавлено 6, max_heap: [2, 1], min_heap: [5, 6], медиана: 3.5
Добавлено 3, max_heap: [3, 1, 2], min_heap: [5, 6], медиана: 3.0
Добавлено 4, max_heap: [3, 1, 2], min_heap: [4, 6, 5], медиана: 3.5


### Задание 3.3: Слияние K отсортированных массивов
Объедините K отсортированных массивов в один отсортированный массив.

In [17]:
import heapq
from typing import List

def merge_k_sorted_arrays(arrays):
    """Слить K отсортированных массивов используя min-heap"""
    # TODO: Используйте кучу для эффективного слияния
    # Подсказка: храните в куче кортежи (значение, индекс_массива, индекс_элемента)
    min_heap = []
    result = []
    
    #инициализация
    for i, arr in enumerate(arrays):
        if arr:
            heapq.heappush(min_heap, (arr[0], i, 0))
            
    #слияние
    while min_heap:
        val, arr_idx, element_idx = heapq.heappop(min_heap)
        result.append(val)
        
        next_element_idx = element_idx + 1
        
        if next_element_idx < len(arrays[arr_idx]):
            next_val = arrays[arr_idx][next_element_idx]
            heapq.heappush(min_heap, (next_val, arr_idx, next_element_idx))
            
    return result

# Тест
arrays = [
    [1, 4, 7],
    [2, 5, 8],
    [3, 6, 9]
]

result = merge_k_sorted_arrays(arrays)
print("Исходные массивы:", arrays)
print("Объединенный массив:", result)

Исходные массивы: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]
Объединенный массив: [1, 2, 3, 4, 5, 6, 7, 8, 9]


### Задание 3.4: Планировщик задач с приоритетами
Реализуйте планировщик задач, где каждая задача имеет приоритет.

In [ ]:
import heapq
from typing import List
class Task:
    def __init__(self, name, priority, duration):
        self.name = name
        self.priority = priority  # Чем больше число, тем выше приоритет
        self.duration = duration

    def __lt__(self, other):
        # Для правильной работы с heapq
        return self.priority > other.priority  # Инвертируем для max-heap поведения

    def __repr__(self):
        return f"Task({self.name}, p={self.priority}, d={self.duration})"

class TaskScheduler:
    def __init__(self):
        # TODO: Инициализируйте кучу для задач
        self.priority_queue: List[Task] = []

    def add_task(self, task):
        """Добавить задачу в планировщик"""
        # TODO: Добавьте задачу в кучу
        heapq.heappush(self.priority_queue, task)
        

    def get_next_task(self):
        """Получить задачу с наивысшим приоритетом"""
        # TODO: Извлеките задачу с максимальным приоритетом
        if self.priority_queue:
            return heapq.heappop(self.priority_queue)
        return None

    def has_tasks(self):
        """Проверить наличие задач"""
        # TODO: Проверьте, есть ли задачи в планировщике
        return len(self.priority_queue) > 0

# Тест планировщика
scheduler = TaskScheduler()
tasks = [
    Task("Email", 2, 5),
    Task("Bug Fix", 5, 20),
    Task("Meeting", 3, 30),
    Task("Code Review", 4, 15),
    Task("Documentation", 1, 10)
]

for task in tasks:
    scheduler.add_task(task)

print("Выполнение задач по приоритету:")
while scheduler.has_tasks():
    task = scheduler.get_next_task()
    print(f"Выполняется: {task}")

## Часть 4: Сравнение производительности

In [18]:
import random
import time
import heapq
from typing import List, Optional

def benchmark_structures(n=10000):
    """Сравнить производительность различных структур для поиска максимума"""

    # Генерация случайных данных
    data = [random.randint(1, n*10) for _ in range(n)]

    # 1. Обычный список (поиск максимума за O(n))
    start = time.perf_counter()
    for _ in range(100):
        max(data)
    list_time = time.perf_counter() - start

    # 2. Отсортированный список (вставка O(n), максимум O(1))
    sorted_list = []
    start = time.perf_counter()
    for val in data[:100]:  # Меньше операций из-за медленности
        sorted_list.append(val)
        sorted_list.sort()
        max_val = sorted_list[-1]
    sorted_time = time.perf_counter() - start

    # 3. Heap (вставка O(log n), максимум O(log n))
    heap = []
    start = time.perf_counter()
    for val in data[:1000]:  # Больше операций
        heapq.heappush(heap, -val)  # Отрицательные для max-heap
        max_val = -heap[0]
    heap_time = time.perf_counter() - start

    print(f"Результаты для {n} элементов:")
    print(f"Список (поиск max): {list_time:.4f}s")
    print(f"Отсортированный список: {sorted_time:.4f}s (100 операций)")
    print(f"Heap: {heap_time:.4f}s (1000 операций)")

benchmark_structures()

Результаты для 10000 элементов:
Список (поиск max): 0.0112s
Отсортированный список: 0.0001s (100 операций)
Heap: 0.0002s (1000 операций)


In [ ]:
#MaxHeap
class MockMaxHeap:
    def __init__(self):
        self.heap = [90, 70, 80, 20, 60, 25, 50]
        
heap = MockMaxHeap() 

def visualize_heap(heap: List[int]):
    # импорт Digraph и List,
    from typing import List
    try:
        from graphviz import Digraph
    except ImportError:
        # Заглушка
        class Digraph:
            def __init__(self, *args, **kwargs):
                pass
            def node(self, *args, **kwargs):
                pass
            def edge(self, *args, **kwargs):
                pass
            def render(self, *args, **kwargs):
                print("[Визуализация: graphviz не установлен или не импортирован]")
                return None

    dot = Digraph()
    for i, val in enumerate(heap):
        dot.node(str(i), str(val))
        
        # 90
        #70 -> par 0
        # 80 -> par 0
        # 20 -> par 1
        # 60 -> par 1
        # 25 -> par 2
        # 50 -> par 2
        
        parent_idx = (i - 1) // 2
        if i > 0:
            dot.edge(str(parent_idx), str(i))
    return dot
print("Визуализация кучи:")
# visualize_heap(heap.heap)  # Раскомментируйте после реализации
viz_output = visualize_heap(heap.heap)
print(viz_output) 

Визуализация кучи:
<__main__.visualize_heap.<locals>.Digraph object at 0x00000202685D32F0>


## Вопросы для отчёта

### По видам деревьев:
1. В чём основное различие между N-арным деревом и бинарным деревом?
2. Почему BST обеспечивает поиск за O(log n) в среднем случае?
3. Какие преимущества даёт Trie при работе со строками?
4. Когда стоит использовать каждый тип дерева?

### По двоичной куче:
5. Какая временная сложность операций `insert` и `extract_max`? Обоснуйте.
6. Почему алгоритм `heapify` работает за O(n), а не за O(n log n)?
7. В чём отличие двоичной кучи от бинарного дерева поиска?

### По применению куч:
8. Почему для задачи "K наибольших элементов" используется min-heap размера K?
9. Как две кучи помогают находить медиану за O(log n)?
10. В чём преимущество использования кучи при слиянии K массивов?
11. Приведите примеры реальных систем, где используются очереди с приоритетом.
12. Сравните эффективность heapsort с быстрой сортировкой на случайных данных.

## Заключение

В этой лабораторной работе вы изучили:
- Различные типы древовидных структур данных
- Реализацию и применение двоичной кучи
- Практические алгоритмы, использующие кучи
- Анализ производительности различных структур данных

Эти знания помогут вам в решении широкого спектра алгоритмических задач и оптимизации производительности программ.